In [1]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np

In [2]:
mp3_head = 'https://www.redkaraoke.com/data/mp3/es/'
mp3_aws_head = 'https://s3.amazonaws.com/redkaraoke/data/mp3/'
mp4_head = 'https://s3.amazonaws.com/redkaraokecom/data/vid/11827/'

In [3]:
links_list = []
for i in range(10):
    r = requests.get('https://www.redkaraoke.com/karaoke/celine-dion/my-heart-will-go-on/11827/recordings/p' + str(i))
    soup = BeautifulSoup(r.content, 'html.parser')
    for link in soup.find_all('a', attrs = {'class':'result result-recording'}):
        links_list.append('https://www.redkaraoke.com' + link.get('href'))
# Remove duplicated links
links_list_unique = set(links_list)
links_list_unique

{'https://www.redkaraoke.com/profile/Arena55/recordings/648309',
 'https://www.redkaraoke.com/profile/BERNITA/recordings/1305787',
 'https://www.redkaraoke.com/profile/Bea18/recordings/624790',
 'https://www.redkaraoke.com/profile/CallieFord/recordings/65921',
 'https://www.redkaraoke.com/profile/Feyth/recordings/61267',
 'https://www.redkaraoke.com/profile/Gracellen/recordings/56971',
 'https://www.redkaraoke.com/profile/ISABEL961/recordings/589633',
 'https://www.redkaraoke.com/profile/Jewel_Emi/recordings/11038',
 'https://www.redkaraoke.com/profile/Karen_Jara/recordings/76253',
 'https://www.redkaraoke.com/profile/Kyra27/recordings/58245',
 'https://www.redkaraoke.com/profile/Musiclover/recordings/533114',
 'https://www.redkaraoke.com/profile/POLLENSINA/recordings/446158',
 'https://www.redkaraoke.com/profile/Ticha/recordings/430226',
 'https://www.redkaraoke.com/profile/Zai_Ara/recordings/1238679',
 'https://www.redkaraoke.com/profile/adr1ana/recordings/448053',
 'https://www.redk

In [4]:
len(links_list_unique)

32

In [5]:
# Features in javascript object "userRecording" :
# features = ['file','title','u','uavatar','ugroup','a','rid','adv']
features1 = ['views','likes','comments','recordings','followers','following','date','gender_loc']
features2 = ['rid','file','u','ugroup']

df = pd.DataFrame(0, index=np.arange(len(links_list_unique)), columns = features1 + features2)
counter = 0

views = []
likes = []
comments = []
recordings = []
followers = []
following = []
date = []
gender_loc = []

# Send request for each link in links_list_unique
for link_sub in links_list_unique:
    r = requests.get(link_sub)
    soup = BeautifulSoup(r.content, 'lxml')
    
    try:
        views.append(int(soup.find('li', attrs = {'class':'rec-views'}).text))
        likes.append(soup.find('li', attrs = {'class':'rec-likes'}).text)
        comments.append(int(soup.find('li', attrs = {'class':'rec-comments'}).text))
        recordings.append(int(soup.find('li', attrs = {'class':'tam1'}).find('strong').text))
        followers.append(int(soup.find_all('ul', attrs = {'class':'stats'})[0].find_all('li')[5].find('strong').text))
        following.append((soup.find_all('ul', attrs = {'class':'stats'})[0].find_all('li')[6].find('strong').text))
        date.append(soup.find_all('div', attrs = {'class':'infoTema_textoBlanco'})[2].text[3:])
        gender_loc.append(soup.find('div', attrs = {'class':'userdata'}).find('p').text)
    except IndexError as e:
        continue
    
    # Localize data in "userRecording" object
    pattern = re.compile(r"userRecording")
    data = soup.find('script', text=pattern)

    for i in features2:
        string_head = 'userRecording.' + i + '='
        try:
            # Take the tail which contains the information needed
            full_string = re.search(string_head + '.*"',str(data)).group(0)
            temp = full_string[len(string_head)+1:-1] 
            df.loc[counter,i] = temp
#             print(temp)
        except AttributeError as e:
            continue

    counter = counter + 1

In [6]:
df.to_csv('./data/out.csv',index=False)

In [7]:
link_sub = links_list[0]
r = requests.get(link_sub)
soup = BeautifulSoup(r.content, 'lxml')

pattern = re.compile(r"userRecording")
data = soup.find('script', text=pattern)

string_head = 'userRecording.u='
full_string = re.search(string_head + '.*"',str(data)).group(0)
user_id = re.sub(r'"', '', full_string.split(string_head)[1])  
print(user_id)

string_head = 'userRecording.rid='
full_string = re.search(string_head + '.*"',str(data)).group(0)
rid = full_string[len(string_head)+1:-1]
print(rid)

string_head = 'userRecording.title='
full_string = re.search(string_head + '.*"',str(data)).group(0)
title = full_string[len(string_head)+1:-1] 
print(title)

string_head = 'userRecording.s3='
full_string = re.search(string_head + '.*"',str(data)).group(0)
s3 = full_string[len(string_head)+1:-1]  
print(s3)

string_head = 'userRecording.file='
full_string = re.search(string_head + '.*"',str(data)).group(0)
link = full_string[len(string_head)+1:-1]
print(link)

Kyra27
58245
My heart will go on
1
11827/4f9784e869afbc597abfba239c9ad122_1.mp3


In [8]:
links_list

['https://www.redkaraoke.com/profile/Kyra27/recordings/58245',
 'https://www.redkaraoke.com/profile/Gracellen/recordings/56971',
 'https://www.redkaraoke.com/profile/Musiclover/recordings/533114',
 'https://www.redkaraoke.com/profile/CallieFord/recordings/65921',
 'https://www.redkaraoke.com/profile/marisabel59/recordings/676158',
 'https://www.redkaraoke.com/profile/chris_hg/recordings/698416',
 'https://www.redkaraoke.com/profile/Jewel_Emi/recordings/11038',
 'https://www.redkaraoke.com/profile/Arena55/recordings/648309',
 'https://www.redkaraoke.com/profile/Karen_Jara/recordings/76253',
 'https://www.redkaraoke.com/profile/Feyth/recordings/61267',
 'https://www.redkaraoke.com/profile/javiernr1024/recordings/589132',
 'https://www.redkaraoke.com/profile/yasumiyo/recordings/297890',
 'https://www.redkaraoke.com/profile/POLLENSINA/recordings/446158',
 'https://www.redkaraoke.com/profile/marypechu/recordings/624481',
 'https://www.redkaraoke.com/profile/mimilove/recordings/605567',
 'ht

In [9]:
link_sub = links_list[0]
r = requests.get(link_sub)
soup = BeautifulSoup(r.content, 'lxml')
soup

<!DOCTYPE html>
<!--[if lt IE 7]>      <html xmlns="https://www.w3.org/1999/xhtml" xmlns:fb="https://www.facebook.com/2008/fbml" class="no-js lt-ie9 lt-ie8 lt-ie7"> <![endif]--><!--[if IE 7]>         <html xmlns="https://www.w3.org/1999/xhtml" xmlns:fb="https://www.facebook.com/2008/fbml" class="no-js lt-ie9 lt-ie8"> <![endif]--><!--[if IE 8]>         <html xmlns="https://www.w3.org/1999/xhtml" xmlns:fb="https://www.facebook.com/2008/fbml" class="no-js lt-ie9"> <![endif]--><!--[if gt IE 8]><!--><html class="no-js" xmlns="https://www.w3.org/1999/xhtml" xmlns:fb="https://www.facebook.com/2008/fbml"> <!--<![endif]-->
<head>
<title>Kyra27 sings My heart will go on in the style of Celine Dion on Red Karaoke</title>
<meta content="Recording of My heart will go on in the style of Celine Dion by Kyra27. Have fun singing and share it with your friends!" name="description"/>
<meta charset="utf-8"/>
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<link href="https://d4rgvppl55fks.

In [10]:
views = int(soup.find('li', attrs = {'class':'rec-views'}).text)
likes = int(soup.find('li', attrs = {'class':'rec-likes'}).text)
comments = int(soup.find('li', attrs = {'class':'rec-comments'}).text)
recordings = int(soup.find('li', attrs = {'class':'tam1'}).find('strong').text)
followers = int(soup.find_all('ul', attrs = {'class':'stats'})[0].find_all('li')[5].find('strong').text)
following = int(soup.find_all('ul', attrs = {'class':'stats'})[0].find_all('li')[6].find('strong').text)
date = soup.find_all('div', attrs = {'class':'infoTema_textoBlanco'})[2].text[3:]
gender_loc = soup.find('div', attrs = {'class':'userdata'}).find('p').text

In [11]:
df['views'] = views
df['likes'] = likes
df['comments'] = comments
df['recordings'] = recordings
df['followers'] = followers
df['following'] = following

In [12]:
df

,views,likes,comments,recordings,followers,following,date,gender_loc,rid,file,u,ugroup
0,7300,231,199,6,273,0,0,0,342009,11827/970e59f114ebdbccca213a39aaec47cd_1.mp3,n1angel4u,9
1,7300,231,199,6,273,0,0,0,722120,11827/cf8281d22d136c250038c34d424da24a_1.mp3,tinarosy,2
2,7300,231,199,6,273,0,0,0,648309,11827/05d2bc50fa9e0ea8a4759bfc0c1be19a_1.mp3,Arena55,2
3,7300,231,199,6,273,0,0,0,342187,11827/d000a7c8203d3922f6d69a3f96d4fdef_1.mp3,n1angel4u,9
4,7300,231,199,6,273,0,0,0,297890,11827/81352dfcf78542cd24af7d8250c73a5a_1.mp4,yasumiyo,9
5,7300,231,199,6,273,0,0,0,589633,11827/0603557c994bdd8b94685a54584cf7b8_1.mp3,ISABEL961,2
6,7300,231,199,6,273,0,0,0,342153,11827/205865feeaa7a1dbdf69e1178608a570_1.mp3,n1angel4u,9
7,7300,231,199,6,273,0,0,0,1159769,11827/27316c7ada6d21c0986a88d5eee10733_1.mp4,guadalupesalinas,2
8,7300,231,199,6,273,0,0,0,76253,11827/2ea24cee588c4b0d7b30b0e1d6b364c6_1.mp3,Karen_Jara,2
9,7300,231,199,6,273,0,0,0,342684,11827/2f69f038e15f5fb8d1e39a91fbd9db5f_1.mp3,n1angel4u,9


In [13]:
gender_loc

'Woman, Trenton, NJ, USA'

In [14]:
date

'06/15/2010'